In [1]:
import pandas as pd

Загрузим выборку языков балканского союза и соотнесем их с пятью языковыми признаками (количество родОв, количество падежей, определенные артикли, типология порядка слов, соотношение согласных-гласных)

In [2]:
balcan_union = pd.read_csv("language.csv")
languages = ['Greek (Modern)', 'Albanian', 'Bulgarian', 'Macedonian', 'Romanian']
balcan_union = balcan_union.loc[balcan_union['Name'].isin(languages)]
balcan_union = balcan_union[['Name','30A Number of Genders', '49A Number of Cases', '37A Definite Articles',
                    '81A Order of Subject, Object and Verb', '3A Consonant-Vowel Ratio']]
balcan_union.reset_index(inplace=True)
balcan_union.drop('index', axis=1, inplace=True)

# добавление пропущенных значений
balcan_union['30A Number of Genders'] = ['3 Three', '3 Three', '3 Three', '3 Three', '3 Three']

balcan_union

,Name,30A Number of Genders,49A Number of Cases,37A Definite Articles,"81A Order of Subject, Object and Verb",3A Consonant-Vowel Ratio
0,Albanian,3 Three,4 4 cases,3 Definite affix,2 SVO,3 Average
1,Bulgarian,3 Three,1 No morphological case-marking,1 Definite word distinct from demonstrative,2 SVO,3 Average
2,Greek (Modern),3 Three,3 3 cases,1 Definite word distinct from demonstrative,7 No dominant order,3 Average
3,Macedonian,3 Three,NaN,1 Definite word distinct from demonstrative,2 SVO,NaN
4,Romanian,3 Three,2 2 cases,3 Definite affix,2 SVO,3 Average


Аналогичное действие проделываем со случайно выбранными европейскими языками 

In [3]:
random_european = pd.read_csv("language.csv")
languages = ['Polish', 'French', 'Bulgarian', 'German', 'English']
random_european = random_european.loc[random_european['Name'].isin(languages)]
random_european = random_european[['Name','30A Number of Genders', '49A Number of Cases', '37A Definite Articles',
                    '81A Order of Subject, Object and Verb', '3A Consonant-Vowel Ratio']]
random_european.reset_index(inplace=True)
random_european.drop('index', axis=1, inplace=True)

# insertion of missing vals
random_european['30A Number of Genders'] = ['3 Three', '3 Three', '2 Two', '3 Three', '3 Three']

random_european


,Name,30A Number of Genders,49A Number of Cases,37A Definite Articles,"81A Order of Subject, Object and Verb",3A Consonant-Vowel Ratio
0,Bulgarian,3 Three,1 No morphological case-marking,1 Definite word distinct from demonstrative,2 SVO,3 Average
1,English,3 Three,2 2 cases,1 Definite word distinct from demonstrative,2 SVO,1 Low
2,French,2 Two,1 No morphological case-marking,1 Definite word distinct from demonstrative,2 SVO,1 Low
3,German,3 Three,4 4 cases,1 Definite word distinct from demonstrative,7 No dominant order,1 Low
4,Polish,3 Three,6 6-7 cases,5 No definite or indefinite article,2 SVO,4 Moderately high


Составим таблицу совместной встречаемости языковых признаков у выбранных языков в языковых подгруппах

In [4]:
two_groups_df = pd.DataFrame({'Group':['Balkan Union','Random European'], '30A':[5,4],
                             '49A':[0,0], '37A':[3,4], '81A':[4,4],
                             '3A':[4,3]})
two_groups_df

,Group,30A,49A,37A,81A,3A
0,Balkan Union,5,0,3,4,4
1,Random European,4,0,4,4,3


Для оценки между двумя малыми выборками воспользуемся U-критерием Манна-Уитни.

U-критерий Манна-Уитни используется для оценки различий между двумя малыми выборками по уровню количественно измеряемого признака. 

Эмпирическое значение U вычисляется по следующей формуле: U = n1 * n2 + (nx*(nx+1)) / 2 - Tx
Нулевая гипотеза H0={уровень признака во второй выборке не ниже уровня признака в первой выборке}; альтернативная гипотеза – H1={уровень признака во второй выборке ниже уровня признака в первой выборке}.


Подробнее об алгоритме применения U-критерия Манна-Уитни: https://goo.gl/HbJr8Q

In [5]:
ranks_df = pd.DataFrame({'Group':['BU', 'RE', 'BU', 'RE', 'RE', 'RE', 'BU', 'RE', 'BU', 'BU'], 'Feature':['49A', '49A', '37A', '3A', '30A', '37A', '81A', '81A', '3A', '30A'],
                        'Selection':[0, 0, 3, 3, 4, 4, 4, 4, 4, 5], 'Rank':[1.5, 1.5, 3.5, 3.5, 7, 7, 7, 7, 7, 10]})


balkan_ranks = ranks_df.loc[ranks_df['Group']=='BU']
rand_european_ranks = ranks_df.loc[ranks_df['Group']=='RE']
balkan_ranks

,Group,Feature,Selection,Rank
0,BU,49A,0,1.5
2,BU,37A,3,3.5
6,BU,81A,4,7.0
8,BU,3A,4,7.0
9,BU,30A,5,10.0


In [6]:
rand_european_ranks

,Group,Feature,Selection,Rank
1,RE,49A,0,1.5
3,RE,3A,3,3.5
4,RE,30A,4,7.0
5,RE,37A,4,7.0
7,RE,81A,4,7.0


In [7]:
Tx = max([sum(balkan_ranks['Rank']), sum(rand_european_ranks['Rank'])])
n1 = len(balkan_ranks)
n2 = len(rand_european_ranks)
nx = max([n1,n2])

u_emp = n1 * n2 + (nx*(nx+1)) / 2 - Tx

u_crit = 4 #Значение из таблицы для p<=0.05

print(f'u эмпирическое = {u_emp}')

u эмпирическое = 11.0


Значение u_emp = 11, что превышает u_crit = 4. Это значит, что эмпирическое значение находится в зоне незначимости.

Гипотеза H0 о незначительности различий между двумя классами принимается, если u_crit < u_emp. В противном случае H0 отвергается и различие определяется как существенное.

Следовательно, различия между совокупностями выбранных признаков для языков балканского союза и для случайно выбранных европейских языков можно считать несущественными.